## Exploring Momementum Strategies

Notes from Ernie Chan's Algo Trading, Winning Strategies and their Rational

Four main causes of momentum:

1. For futures, the persistence of roll returns, especially of their signs.
2. The slow diffusion, analysis,, and acceptance of new information
3. The forced sales or purchases of assets of various types of funds
4. Market manipulation by high frequenc traders

Researchers sometimes classify momentum in asset prices into two types:

- Time Series Momentum. Past returns are correlated with future returns
- Cross Sectional Momentum. Relative performance of a price series in relation to another price series. ie prices with returns that outperformed other price series will likely keep doing so in the future.

### Tests for Time Series Momentum 

We should find the optimal pair of past and future periods that gives the highest positie correlation and use that as our look back and holding period for the momentum strategy

If we are interested instead in finding out whether there is long-term trending behavior in the time series without regard to specific time frames, we can calculate the Hurst exponent together with the Variance Ratio test to rule out the null hypothesis of random walk

In [70]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import ccxt

lookback_values = [1, 5, 10, 25, 60, 120, 250]
hold_values = [1, 5, 10, 25, 60, 120, 250]
symbol = 'BTC/USDT'
exchange = ccxt.binance()

# Fetch Prices
price_data = exchange.fetch_ohlcv(symbol, '1h', limit=1000)
price_df = pd.DataFrame(price_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Add Date
price_df['date'] = pd.to_datetime(price_df['timestamp'], unit="ms")

# Calculate Log Returns
price_df['returns'] = np.log(price_df['close'] / price_df['close'].shift(1))
returns = price_df['returns'].copy()


close_prices = price_df['close']
price_df.to_csv('data.csv')

correlation_results = []

for lb in lookback_values:
    for hd in hold_values:
        ret_lag = (close_prices - close_prices.shift(lb)) / close_prices.shift(lb)
        ret_fut = (close_prices.shift(hd) - close_prices) / close_prices

        badDates = np.any([ret_lag.isna(), ret_fut.isna()], axis=0)
        ret_lag = ret_lag[~badDates]
        ret_fut = ret_fut[~badDates]

        if lb >= hd:
            indepSet = np.arange(0, len(ret_lag), lb)
            print("ind set: ", indepSet)
        else:
            indepSet = np.arange(0, len(ret_lag), hd)

        ret_lag = ret_lag.iloc[indepSet]
        ret_fut = ret_fut.iloc[indepSet]


        print("Lag", ret_lag)
        print('Future: ', ret_fut)

        cc, pval = pearsonr(ret_lag, ret_fut)

        correlation_results.append({
            'lookback': lb,
            'holding': hd,
            'correlation': cc,
            'pvalue': pval
        })

result = pd.DataFrame(data=correlation_results)
result.to_csv("result.csv")

ind set:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 